In [1]:
import random
from bs4 import BeautifulSoup
import requests
import pandas as pd

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_name_dict = {'ATL': 'Atlanta',
                        'WSN': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SFG': 'San Francisco',
                        'COL': 'Colorado',
                        'SDP': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TBR': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KCR': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
acronym_to_name_dict = {'ATL': 'Atlanta'}

In [2]:
class Player:
    """Class that holds information for a player.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.singles = singles
        self.doubles = singles + doubles
        self.triples = singles + doubles + triples
        self.home_runs = singles + doubles + triples + home_runs
        self.walks = singles + doubles + triples + home_runs + walks
        self.cum_singles = 0
        self.cum_doubles = 0
        self.cum_triples = 0
        self.cum_home_runs = 0
        self.cum_walks = 0
        self.cum_pa = 0
        self.cum_runs = 0
        self.cum_rbi = 0
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if(self.cum_pa-self.cum_walks>0):
            return(float((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs)/(self.cum_pa-self.cum_walks)))
        else:
            return 0    
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa>0):
            return((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs+self.cum_walks)/(self.cum_pa))
        else:
            return 0
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if(self.cum_pa-self.cum_walks>0):
            return((self.cum_singles+2*self.cum_doubles+3*self.cum_triples+4*self.cum_home_runs)/(self.cum_pa-self.cum_walks))
        else:
            return 0
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str(self.cum_home_runs))

In [3]:
class Team:
    """Class that holds information for a team.
    
    Attributes:
        name: team name
        p1-p9: instances of player class
        roster: batting lineup of team in list form
        index: position in batting order of current batter
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, name, p1, p2, p3, p4, p5, p6, p7, p8, p9):
        """Initializes values for this class"""
        self.name = name
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.p5 = p5
        self.p6 = p6
        self.p7 = p7
        self.p8 = p8
        self.p9 = p9
        self.roster = [p1, p2, p3, p4, p5, p6, p7, p8, p9]
        self.index = 0
        self.runners = [None, None, None, 0]    # first base-hom plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        self.index = 0
        self.runners = [None, None, None, 0]
        self.runs = 0
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]

In [4]:
class Game:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.home_team = home_team
        self.away_team = away_team
    def get_result(self, player, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            player: instance of a player class
            num: random number between 0 and 1
        Returns:
            str: refers to result based on random number (ie "walk")
        """
        player.cum_pa+=1
        
        if(num<player.singles):
            player.cum_singles+=1
            return "single"
        elif(num<player.doubles):
            player.cum_doubles+=1
            return "double"
        elif(num<player.triples):
            player.cum_triples+=1
            return "triple"
        elif(num<player.home_runs):
            player.cum_home_runs+=1
            return "home_run"
        elif(num<player.walks):
            player.cum_walks+=1
            return "walk"
        else:
            return "out"  
    def move_runners(self, team, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            result: str of reult for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.roster[team.index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.roster[team.index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.roster[team.index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.roster[team.index].cum_runs+=1
            team.roster[team.index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.roster[team.index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0     
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + away_team.name + " " + str(away_team.runs) + " " + home_team.name + " " + str(home_team.runs))     
    def print_record(self):
        """Printed score of a game""" 
        print(away_team.name + ": " + str(away_team.wins) + "-" + str(away_team.losses))
        print(home_team.name + ": " + str(home_team.wins) + "-" + str(home_team.losses))     
    def play(self):
        """A game between two instances of the team class is played""" 
        inning = 1
        while(inning<10 or away_team.runs==home_team.runs):
            for side in [away_team, home_team]:
                outs = 0

                while(outs<3):
                    num = random.uniform(0,1)
                    result = self.get_result(side.roster[side.index], num)
                    if(result=="out"):
                        outs+=1
                    else:
                        self.move_runners(side, result)
                        
                    side.index+=1
                    side.index = side.index%8
                    
                side.reset_runners()
            inning+=1
        
        if(away_team.runs>home_team.runs):
            away_team.wins+=1
            home_team.losses+=1
        elif(away_team.runs<home_team.runs):
            away_team.losses+=1
            home_team.wins+=1
        
        away_team.cum_runs+=away_team.runs
        away_team.restart()
        home_team.cum_runs+=home_team.runs
        home_team.restart()              

In [5]:
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.home_team = home_team
        self.away_team = away_team
    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(away_team, home_team)
            g.play() 

In [6]:
a1 = Player("Away", "1", .170, .080, .010, .030, .050)
a2 = Player("Away", "2", .170, .080, .010, .030, .050)
a3 = Player("Away", "3", .170, .080, .010, .030, .050)
a4 = Player("Away", "4", .170, .080, .010, .030, .050)
a5 = Player("Away", "5", .170, .080, .010, .030, .050)
a6 = Player("Away", "6", .170, .080, .010, .030, .050)
a7 = Player("Away", "7", .170, .080, .010, .030, .050)
a8 = Player("Away", "8", .170, .080, .010, .030, .050)
a9 = Player("Away", "9", .170, .080, .010, .030, .050)
h1 = Player("Home", "1", .170, .080, .010, .030, .050)
h2 = Player("Home", "2", .170, .080, .010, .030, .050)
h3 = Player("Home", "3", .170, .080, .010, .030, .050)
h4 = Player("Home", "4", .170, .080, .010, .030, .050)
h5 = Player("Home", "5", .170, .080, .010, .030, .050)
h6 = Player("Home", "6", .170, .080, .010, .030, .050)
h7 = Player("Home", "7", .170, .080, .010, .030, .050)
h8 = Player("Home", "8", .170, .080, .010, .030, .050)
h9 = Player("Home", "9", .170, .080, .010, .030, .050)
away_team = Team("Away", a1, a2, a3, a4, a5, a6, a7, a8, a9)
home_team = Team("Home", h1, h2, h3, h4, h5, h6, h7, h8, h9)

s = Season(away_team, home_team)
s.simulate_season(1000)
s.print_record()
a1.print_stats()

Away: 524-476
Home: 476-524
Avg: 0.301
OBP: 0.334
SLG: 0.504
OPS: 0.837
HR: 179


In [9]:
def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)

In [10]:
string_columns = ['Pos', 'Name']
int_columns = ['Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
               'RBI', 'SB', 'CS', 'BB', 'SO', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
float_columns = ['BA', 'OBP', 'SLG', 'OPS']

for k, df in batting_dict.items():
    for column in df.columns:
        if(column in int_columns):
            df[column] = df.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            df[column] = df.apply(lambda row: convert_float(row, column), axis=1)

In [29]:
headers = {'User-Agent': header_name}
batting_dict = {}
URL_dict = {v: 'https://www.baseball-reference.com/teams/'+ k + '/2019.shtml' for k, v in acronym_to_name_dict.items()}
data_columns = ['Pos', 'pos', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']
br_link_start = 'https://www.baseball-reference.com'


for name, URL in URL_dict.items():
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    batting_columns = []
    
    for table in soup.findAll('table', attrs={'id': 'team_batting'}):
        for labels in table.findAll('thead'):
            for header_value in labels.findAll('th'):
                if(header_value.get_text() in data_columns):
                    batting_columns.append(header_value.get_text())

                    if(header_value.get_text()=='Name'):  #additional column for player url
                        batting_columns.append('Link')

            df = pd.DataFrame(columns=batting_columns)
            
        for player in table.findAll('tbody'):
            for row in player.findAll('tr'):
                if(row.attrs and row['class'][0]=='thead'):
                    continue

                player_values = []


                for column_value in row.findAll('td'):
                    
                    if(column_value['data-stat'] in data_columns):
                        player_values.append(column_value.get_text())
                        
                        if(column_value['data-stat']=='player'):
                            player_values.append(br_link_start + column_value.a['href'])  #adding player url
                   
                df = df.append(pd.Series(player_values, index=df.columns),
                                                      ignore_index=True)
                
    #df.drop('Rk', axis=1, inplace=True)             
    batting_dict.update({name: df})
        
    print(batting_dict)

{'Atlanta':    Pos                Name                                               Link  \
0    C       Tyler Flowers  https://www.baseball-reference.com/players/f/f...   
1   1B    Freddie Freeman*  https://www.baseball-reference.com/players/f/f...   
2   2B       Ozzie Albies#  https://www.baseball-reference.com/players/a/a...   
3   SS      Dansby Swanson  https://www.baseball-reference.com/players/s/s...   
4   3B      Josh Donaldson  https://www.baseball-reference.com/players/d/d...   
5   LF        Austin Riley  https://www.baseball-reference.com/players/r/r...   
6   CF    Ronald Acuna Jr.  https://www.baseball-reference.com/players/a/a...   
7   RF      Nick Markakis*  https://www.baseball-reference.com/players/m/m...   
8    C       Brian McCann*  https://www.baseball-reference.com/players/m/m...   
9   IF      Johan Camargo#  https://www.baseball-reference.com/players/c/c...   
10  RF      Matthew Joyce*  https://www.baseball-reference.com/players/j/j...   
11  CF     Ender

In [36]:
headers = {'User-Agent': header_name}
pitching_dict = {}
URL_dict = {v: 'https://www.baseball-reference.com/teams/'+ k + '/2019.shtml' for k, v in acronym_to_name_dict.items()}

data_columns = ['Pos', 'pos', 'Name', 'player', 'Age', 'age', 'G', 'GS', 'CG', 'SV', 'H', 'R',
               'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'batters_faced', 'BK', 'WP', 'BF', 'ERA', 
                'earned_run_avg', 'IP', 'FIP', 'fip', 'WHIP', 'whip']

br_link_start = 'https://www.baseball-reference.com'

for name, URL in URL_dict.items():
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    pitching_columns = []
    
    for table in soup.findAll('table', attrs={'id': 'team_pitching'}):
        for labels in table.findAll('thead'):
            for header_value in labels.findAll('th'):
                if(header_value.get_text() in data_columns):
                    pitching_columns.append(header_value.get_text())
                    
                    if(header_value.get_text()=='Name'):  #additional column for player url
                        pitching_columns.append('Link')

            df = pd.DataFrame(columns=pitching_columns)
            
        for player in table.findAll('tbody'):
            for row in player.findAll('tr'):
                if(row.attrs and row['class'][0]=='thead'):
                    continue

                player_values = []

                for column_value in row.findAll('td'):
                    
                    if(column_value['data-stat'] in data_columns):
                        player_values.append(column_value.get_text())
                        
                        if(column_value['data-stat']=='player'):
                            player_values.append(br_link_start + column_value.a['href'])  #adding player url
                            
                df = df.append(pd.Series(player_values, index=df.columns),
                                                      ignore_index=True)
                
    pitching_dict.update({name: df})
        
    print(pitching_dict)

{'Atlanta':    Pos               Name                                               Link  \
0   SP        Mike Soroka  https://www.baseball-reference.com/players/s/s...   
1   SP      Julio Teheran  https://www.baseball-reference.com/players/t/t...   
2   SP         Max Fried*  https://www.baseball-reference.com/players/f/f...   
3   SP   Mike Foltynewicz  https://www.baseball-reference.com/players/f/f...   
4   SP    Dallas Keuchel*  https://www.baseball-reference.com/players/k/k...   
5   SP      Kevin Gausman  https://www.baseball-reference.com/players/g/g...   
6   CL       Luke Jackson  https://www.baseball-reference.com/players/j/j...   
7   RP        Josh Tomlin  https://www.baseball-reference.com/players/t/t...   
8   RP      Sean Newcomb*  https://www.baseball-reference.com/players/n/n...   
9   RP    Anthony Swarzak  https://www.baseball-reference.com/players/s/s...   
10  RP     Jerry Blevins*  https://www.baseball-reference.com/players/b/b...   
11        Touki Toussaint  h

In [107]:
string_columns = ['Pos', 'Name']
int_columns = ['Age', 'W', 'L', 'G', 'GS', 'GF', 'CG', 'SHO', 'SV', 'H', 'R',
               'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+']
float_columns = ['W-L%', 'ERA', 'IP', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9',
                'SO/W']

for k, df in pitching_dict.items():
    for column in df.columns:
        if(column in int_columns):
            df[column] = df.apply(lambda row: convert_int(row, column), axis=1)
        elif(column in float_columns):
            df[column] = df.apply(lambda row: convert_float(row, column), axis=1)

.765
.476
.739
.571
.500
.300
.818
.667
.667
.333
1.000
1.000
1.000
.429
.000
.750
1.000
.500
.000
.500
.333
.000
.000
1.000
2.68
3.81
4.02
4.54
3.75
6.19
3.84
3.74
3.16
4.31
3.90
5.62
1.39
7.06
6.21
4.01
4.98
3.86
7.20
8.69
4.08
3.52
3.00
5.40
9.64
1.93
1.69
17.36
2.25
18.00
0.00
174.2
174.2
165.2
117.0
112.2
80.0
72.2
79.1
68.1
39.2
32.1
41.2
32.1
29.1
29.0
24.2
21.2
21.0
20.0
19.2
17.2
15.1
12.0
11.2
9.1
9.1
5.1
4.2
4.0
3.0
2.0
0.0


ValueError: invalid literal for int() with base 10: ''

In [57]:
headers = {'User-Agent': header_name}
url_dict = {}
batter_dict = {}
data_columns = ['Year', 'Tm', 'team_ID', 'Pos', 'POS', 'pos_season', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 
                'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']

for k, v in batting_dict.items():
    for name, url in zip(v.Name, v.Link):
        url_dict.update({name: url})
      
for name, URL in url_dict.items():       
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    batting_columns = []
    player_name = soup.find('h1', attrs={'itemprop': 'name'}).get_text()
    print(player_name)
    
    for table in soup.findAll('table', attrs={'id': 'batting_standard'}):
        for labels in table.findAll('thead'):
            for header_value in labels.findAll('th'):
                if(header_value.get_text() in data_columns):
                    batting_columns.append(header_value.get_text())

            df = pd.DataFrame(columns=batting_columns)
            
        for player in table.findAll('tbody'):
            for row in player.findAll('tr'):
                
                if(row['class'][0]=='minors_table'):
                    continue
                    
        
                player_values = []
                player_values.append(row.th.get_text())

                for column_value in row.findAll('td'):
                    
                    if(column_value['data-stat'] in data_columns):
                        player_values.append(column_value.get_text())
                    
                df = df.append(pd.Series(player_values, index=df.columns),
                                                      ignore_index=True)

    batter_dict.update({player_name: df})
    print(df)

    

Tyler Flowers
    Year Age   Tm    G   PA   AB   R   H  2B 3B  HR SB CS  BB   SO HBP    Pos
0   2009  23  CHW   10   20   16   3   3   1  0   0  0  0   3    8   1    /2D
1   2010  24  CHW    8   15   11   2   1   0  0   0  0  0   4    5   0     /2
2   2011  25  CHW   38  129  110  13  23   5  1   5  0  1  14   38   3   2/3D
3   2012  26  CHW   52  153  136  19  29   6  0   7  2  1  12   56   4   2/3D
4   2013  27  CHW   84  275  256  24  50  11  0  10  0  1  14   94   4      2
5   2014  28  CHW  127  442  407  42  98  16  1  15  0  1  25  159   8     *2
6   2015  29  CHW  112  361  331  21  79  12  0   9  0  1  21  104   6  *2/3D
7   2016  30  ATL   83  325  281  27  76  18  0   8  0  0  29   91  11      2
8   2017  31  ATL   99  370  317  41  89  16  0  12  0  1  31   82  20    2/D
9   2018  32  ATL   82  296  251  34  57   9  0   8  0  0  35   76   9    2/D
10  2019  33  ATL   85  310  271  36  62  11  3  11  0  0  31  105   6      2
Freddie Freeman
   Year Age   Tm    G   PA   AB   

Ender Inciarte
   Year Age   Tm    G   PA   AB   R    H  2B 3B  HR  SB  CS  BB  SO HBP    Pos
0  2014  23  ARI  118  447  418  54  116  18  2   4  19   3  25  53   0  *87/9
1  2015  24  ARI  132  561  524  73  159  27  5   6  21  10  26  58   4   *978
2  2016  25  ATL  131  578  522  85  152  24  7   3  16   7  45  68   4    *87
3  2017  26  ATL  157  718  662  93  201  27  5  11  22   9  49  94   0     *8
4  2018  27  ATL  156  660  597  83  158  27  6  10  28  14  49  86   6     *8
5  2019  28  ATL   65  230  199  30   49  11  2   5   7   1  26  41   4      8
Charlie Culberson
   Year Age   Tm    G   PA   AB   R   H  2B 3B  HR SB CS  BB  SO HBP       Pos
0  2012  23  SFG    6   23   22   0   3   0  0   0  0  0   0   7   0        /4
1  2013  24  COL   47  104   99  12  29   5  0   2  5  1   4  23   0       7/4
2  2014  25  COL   95  233  210  17  41   7  2   3  2  2  12  62   5     564/3
3                                                                             
4  2016  27  LAD   

Julio Teheran
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3  0  0  12  43   1  2/D
3  2016  25  MIN  26   90   82   4  12  3  0  1  0  0   5  19   0    2
4  2017  26  ARI   5    7    7   0   1  1  0  0  0  0   0   1   0   /2
5  2018  27  ARI  87  223  208  19  42  9  0  9  0  0  11  71   1  2/D
6  2019  28  TOT  26   70   63   9  11  3  0  4  0  0   6  28   0  2/1
7  2019  28  ARI  25   69   62   9  11  3  0  4  0  0   6  28   0  2/1
8  2019  28  ATL   1    1    1   0   0  0  0  0  0  0   0   0   0   /2
Dallas Keuchel
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3 

Shane Carle
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3  0  0  12  43   1  2/D
3  2016  25  MIN  26   90   82   4  12  3  0  1  0  0   5  19   0    2
4  2017  26  ARI   5    7    7   0   1  1  0  0  0  0   0   1   0   /2
5  2018  27  ARI  87  223  208  19  42  9  0  9  0  0  11  71   1  2/D
6  2019  28  TOT  26   70   63   9  11  3  0  4  0  0   6  28   0  2/1
7  2019  28  ARI  25   69   62   9  11  3  0  4  0  0   6  28   0  2/1
8  2019  28  ATL   1    1    1   0   0  0  0  0  0  0   0   0   0   /2
Jeremy Walker
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3  0 

Arodys Vizcaíno
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3  0  0  12  43   1  2/D
3  2016  25  MIN  26   90   82   4  12  3  0  1  0  0   5  19   0    2
4  2017  26  ARI   5    7    7   0   1  1  0  0  0  0   0   1   0   /2
5  2018  27  ARI  87  223  208  19  42  9  0  9  0  0  11  71   1  2/D
6  2019  28  TOT  26   70   63   9  11  3  0  4  0  0   6  28   0  2/1
7  2019  28  ARI  25   69   62   9  11  3  0  4  0  0   6  28   0  2/1
8  2019  28  ATL   1    1    1   0   0  0  0  0  0  0   0   0   0   /2
Huascar Ynoa
   Year Age   Tm   G   PA   AB   R   H 2B 3B HR SB CS  BB  SO HBP  Pos
0  2013  22  NYY  16   27   26   3   4  1  0  0  0  0   1   9   0    2
1  2014  23  NYY  32   85   81   7  23  4  0  1  0  0   4  22   0  2/D
2  2015  24  NYY  67  172  155  21  43  9  1  3 